In [8]:
import numpy as np
import xarray as xr
import pandas as pd
from scipy.ndimage import label
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap
from matplotlib.animation import FuncAnimation, PillowWriter
import pickle
from IPython.display import HTML

In [2]:
def read_rgb_file(file_path):
    rgb_values = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip() and not line.startswith('#') and all(c.isdigit() for c in line.split()):
                rgb_values.append([int(c) for c in line.split()])
    return np.array(rgb_values)

rgb_file_path = '/Users/lilydonaldson/Downloads/examples/util/colormap/precip_11lev.rgb'

# Read the RGB file and create a custom colormap
rgb_values = read_rgb_file(rgb_file_path)
rgb_values = rgb_values / 255.0  # Normalize the RGB values to [0, 1]
custom_cmap = ListedColormap(rgb_values)

In [3]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

def find_and_plot_closest_closed_slp_loop(slp_data, storm_center_lat, storm_center_lon, precipitation_array, threshold=20):
    latitudes = slp_data['latitude'].values
    longitudes = slp_data['longitude'].values
    slp = slp_data['msl'].values

    # Check if the storm center is out of bounds
    if storm_center_lon < -85 or storm_center_lon > -67:
        print(f"Storm center {storm_center_lon} out of bounds.")
        return np.full(precipitation_array.shape, np.nan), None, None, None, None, None

    # Convert latitudes and longitudes to 2D grid if they are 1D
    if latitudes.ndim == 1 and longitudes.ndim == 1:
        latitudes, longitudes = np.meshgrid(latitudes, longitudes, indexing='ij')

    distances = np.array([[haversine(storm_center_lat, storm_center_lon, lat, lon) for lon in longitudes[0]] for lat in latitudes[:, 0]])
    min_dist_idx = np.unravel_index(np.argmin(distances), distances.shape)
    closest_lat = latitudes[min_dist_idx]
    closest_lon = longitudes[min_dist_idx]

    structure = np.ones((3, 3), dtype=bool)
    labeled, num_features = label(slp < np.percentile(slp, threshold), structure=structure)

    centroids = np.array([np.mean(np.column_stack(np.where(labeled == i)), axis=0) for i in range(1, num_features + 1)])
    centroids_lat_lon = np.array([(latitudes[int(c[0]), int(c[1])], longitudes[int(c[0]), int(c[1])]) for c in centroids])

    containing_loops = []
    for i in range(1, num_features + 1):
        loop_mask = labeled == i
        loop_latitudes = latitudes[np.where(loop_mask)]
        loop_longitudes = longitudes[np.where(loop_mask)]
        if (storm_center_lat >= loop_latitudes.min()) and (storm_center_lat <= loop_latitudes.max()) and \
           (storm_center_lon >= loop_longitudes.min()) and (storm_center_lon <= loop_longitudes.max()):
            containing_loops.append(i)

    if not containing_loops:
        print("No closed loops contain the storm center.")
        return None, None, None, None, None, None

    closest_loop_idx = min(containing_loops, key=lambda i: haversine(storm_center_lat, storm_center_lon, centroids_lat_lon[i-1][0], centroids_lat_lon[i-1][1]))
    closest_centroid = centroids_lat_lon[closest_loop_idx-1]

    loop_mask = labeled == closest_loop_idx
    lat_diff = np.abs(np.diff(latitudes[:, 0])).mean() * (np.pi / 180) * 6371
    lon_diff = np.abs(np.diff(longitudes[0, :])).mean() * (np.pi / 180) * 6371
    loop_area = np.sum(loop_mask) * lat_diff * lon_diff

    # Check if the loop crosses the specified boundaries
    loop_lat_min, loop_lat_max = latitudes[np.where(loop_mask)].min(), latitudes[np.where(loop_mask)].max()
    loop_lon_min, loop_lon_max = longitudes[np.where(loop_mask)].min(), longitudes[np.where(loop_mask)].max()
    if loop_lon_min < -90 or loop_lon_max > -50 or loop_lat_max > 55 or loop_lat_min < 20:
        print(f"SLP loop crosses boundary: lon({loop_lon_min}, {loop_lon_max}), lat({loop_lat_min}, {loop_lat_max}).")
        return np.full(precipitation_array.shape, np.nan), None, None, None, None, None

    mask = np.full(precipitation_array.shape, np.nan)
    for i in range(loop_mask.shape[0]):
        for j in range(loop_mask.shape[1]):
            lat_idx = np.abs(precipitation_array['latitude'].values - latitudes[i, j]).argmin()
            lon_idx = np.abs(precipitation_array['longitude'].values - longitudes[i, j]).argmin()
            if loop_mask[i, j]:
                mask[lat_idx, lon_idx] = 1

    return mask, closest_lat, closest_lon, longitudes, latitudes, loop_mask

def process_storm_data(storm_data, slp_file_path, precip_file_path, precip_variable='tp', threshold=40):
    storm_library = {}
    mask_library = {}
    slp_data = xr.open_dataset(slp_file_path)
    precip_data = xr.open_dataset(precip_file_path)

    for storm in storm_data:
        storm_id = storm['storm_id']
        start_time = storm['start_time']
        end_time = storm['end_time']
        timesteps = pd.date_range(start=start_time, end=end_time, freq='6H')
        
        start_timestamp = start_time.timestamp()
        end_timestamp = end_time.timestamp()
        
        for timestep in timesteps:
            try:
                precip_var = precip_data[precip_variable].sel(time=timestep, method='nearest')
                scale_factor = precip_var.attrs.get('scale_factor', 1.0)
                add_offset = precip_var.attrs.get('add_offset', 0.0)
                precipitation_array = (precip_var * scale_factor + add_offset) * 1000  # Convert to mm per hour
                
                timestep_timestamp = timestep.timestamp()
                storm_center_lat = np.interp(timestep_timestamp, [start_timestamp, end_timestamp], [storm['start_lat'], storm['end_lat']])
                storm_center_lon = np.interp(timestep_timestamp, [start_timestamp, end_timestamp], [storm['start_lon'], storm['end_lon']])

                mask, closest_lat, closest_lon, slp_lons, slp_lats, loop_mask = find_and_plot_closest_closed_slp_loop(
                    slp_data.sel(time=timestep, method='nearest'), storm_center_lat, storm_center_lon, precipitation_array, threshold)

                if mask is not None:
                    masked_precipitation_array = precipitation_array * mask
                    storm_library[timestep] = masked_precipitation_array
                    mask_library[timestep] = (slp_lons, slp_lats, loop_mask)
                else:
                    storm_library[timestep] = np.full(precipitation_array.shape, np.nan)  # Return empty if no valid loop
                    mask_library[timestep] = (None, None, None)

            except Exception as e:
                print(f"Error processing {timestep}: {e}")

    slp_data.close()
    precip_data.close()

    return storm_library, mask_library

In [4]:
# Load the specific storm data for the year 2010
year = 2010
with open(f"/Users/lilydonaldson/Downloads/examples/data/merra2fronts/identified_bomb_cyclones/updatedJuly22/eastCoast/ERA5_ERA5ar_east_ETCs_{year}.pkl", 'rb') as f:
    year_dataset = pickle.load(f)

# Filter the dataset to include only the storm with the specified storm_id
specific_storm_id = '20101225060600026700'
filtered_year_dataset = [storm for storm in year_dataset if storm['storm_id'] == specific_storm_id]

slp_file_path = "/Volumes/SSK Drive /ERA5_prec/ERA5_SLP_2010.nc"
precip_file_path = "/Volumes/SSK Drive /ERA5_prec/ERA5_prec_2010.nc"

# Process the storm data
storm_library, mask_library = process_storm_data(filtered_year_dataset, slp_file_path, precip_file_path)

/var/folders/3x/6dh1l7m535n5hvm8c7tjlk_00000gn/T/ipykernel_62287/217736227.py:83: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  timesteps = pd.date_range(start=start_time, end=end_time, freq='6H')


Storm center -92.72000000000003 out of bounds.
Storm center -92.35754901960787 out of bounds.
Storm center -90.18284313725492 out of bounds.
Storm center -88.00813725490198 out of bounds.
Storm center -85.83343137254904 out of bounds.
SLP loop crosses boundary: lon(-171.5, -1.5), lat(27.0, 82.5).
SLP loop crosses boundary: lon(-171.5, -1.25), lat(28.25, 82.75).
SLP loop crosses boundary: lon(-180.0, -1.5), lat(27.75, 83.5).
Storm center -66.26107843137255 out of bounds.
Storm center -64.08637254901961 out of bounds.
Storm center -61.911666666666676 out of bounds.
Storm center -59.73696078431373 out of bounds.
Storm center -57.562254901960785 out of bounds.


In [5]:
def plot_masked_precip(ax, timestep, masked_precip, slp_lons, slp_lats, loop_mask, vmin, vmax):
    ax.clear()
    ax.set_extent([-90, -50, 20, 60], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS)
    
    # Plot masked precipitation
    im = masked_precip.plot(ax=ax, transform=ccrs.PlateCarree(), cmap=custom_cmap, vmin=vmin, vmax=vmax, add_colorbar=False)
    
    # Outline the storm boundary in red
    if loop_mask is not None:
        ax.contour(slp_lons, slp_lats, loop_mask, colors='red', linewidths=1.5, transform=ccrs.PlateCarree())
    
    ax.set_title(f'Masked Precipitation for {timestep}')
    return im

In [ ]:
timesteps = list(storm_library.keys())
masked_precip_data = [storm_library[t] for t in timesteps]
slp_lons, slp_lats, loop_masks = zip(*[mask_library[t] for t in timesteps])

vmin = 0
vmax = 10

fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
fig.dpi = 100

initial_plot = plot_masked_precip(ax, timesteps[0], masked_precip_data[0], slp_lons[0], slp_lats[0], loop_masks[0], vmin, vmax)
cbar = fig.colorbar(initial_plot, ax=ax, orientation='vertical', pad=0.02, aspect=50)
cbar.set_label('Precipitation (mm/hour)')

def update(frame):
    timestep = timesteps[frame]
    masked_precip = masked_precip_data[frame]
    slp_lon = slp_lons[frame]
    slp_lat = slp_lats[frame]
    loop_mask = loop_masks[frame]
    ax.clear()
    im = plot_masked_precip(ax, timestep, masked_precip, slp_lon, slp_lat, loop_mask, vmin, vmax)
    return im

ani = FuncAnimation(fig, update, frames=len(timesteps), repeat=True, blit=False)

# Save the animation as a GIF
output_path = 'storm_animation.gif'
writer = PillowWriter(fps=2)
ani.save(output_path, writer=writer)

#display the animation in the jupyter notebook
HTML(ani.to_jshtml())